In [75]:
import os
import sys
import pandas as pd
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

<h1>Loading database</h1>

In [76]:
from pypots.benchpots.datasets import preprocess_physionet2012
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1)

2024-10-30 20:17:43 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2024-10-30 20:17:43 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2024-10-30 20:17:43 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2024-10-30 20:17:43 [INFO]: Loaded successfully!


<h2>Training data</h2>

<h3>Loading training dataset</h3>

In [77]:
train_X = physionet2012_dataset['train_X']

<h3>Divided into subgroups by gender and showing the percentage</h3>

In [78]:
distribution_gender_training = train_X['Gender'].value_counts(normalize=True) * 100
distribution_gender_training # 0: female, or 1: male

Gender
 1.0    56.211706
 0.0    43.697041
-1.0     0.091253
Name: proportion, dtype: float64

<h3>Divided into subgroups by ICUType and showing the percentage</h3>

In [79]:
distribution_ICUType_training = train_X['ICUType'].value_counts(normalize=True) * 100
distribution_ICUType_training

ICUType
3.0    35.445183
4.0    28.288359
2.0    21.170643
1.0    15.095815
Name: proportion, dtype: float64

<h3>Filtering only one age per patient</h3>

In [80]:
age_uniques_train = train_X.groupby('RecordID')['Age'].first().reset_index()
age_uniques_train.head()

,RecordID,Age
0,132539,54.0
1,132541,44.0
2,132543,68.0
3,132548,68.0
4,132554,64.0


<h3>Showing total occurrences of age</h3>

In [81]:
# 60% de 11988 (training set)
age_uniques_train.count()

RecordID    7671
Age         7671
dtype: int64

In [82]:
#first_values = train_X['Age'][::48] #first value of "Age" at every 48th index
#total_quantity_of_age_occurrences = first_values.count()
#total_quantity_of_age_occurrences

In [83]:
#first_values.head()

<h3>Dividing age into subgroup of people aged 65+</h3>

In [84]:
more_than_or_equal_to_65_training = age_uniques_train[age_uniques_train['Age'] >= 65]
# more_than_or_equal_to_65 = train_X[train_X['Age'] >= 65] 
#distribution_age_group_one = more_than_or_equal_to_65['Age'].count() 
#distribution_age_group_one
# age_uniques_more_than_or_equal_to_65 = more_than_or_equal_to_65.groupby('RecordID')['Age'].first().reset_index()
percentage_group_one_train = (more_than_or_equal_to_65_training.count() / age_uniques_train.count())*100
percentage_group_one_train

RecordID    54.803807
Age         54.803807
dtype: float64

<h3>Dividing age into subgroup of people under 65 years of age</h3>

In [85]:
less_than_65 = age_uniques_train[age_uniques_train['Age'] < 65]
# less_than_65 = train_X[train_X['Age'] < 65] 
#distribution_age_group_two = less_than_65['Age'].value_counts(normalize=True) * 100
#distribution_age_group_two

# age_uniques_less_than_65 = less_than_65.groupby('RecordID')['Age'].first().reset_index()
percentage_group_two_train = (less_than_65.count() / age_uniques_train.count()) * 100
percentage_group_two_train

RecordID    45.196193
Age         45.196193
dtype: float64

<h3>Filtering only one height and one weight per patient</h3>

In [86]:
filtered_train_X = train_X[(train_X['Height'] != -1) & (train_X['Weight'] != -1) & (train_X['Height'].notna()) & (train_X['Weight'].notna())] 

In [87]:
height_uniques_train = filtered_train_X.groupby('RecordID')['Height'].first().reset_index() 
height_uniques_train.head()

,RecordID,Height
0,132543,180.3
1,132548,162.6
2,132570,170.2
3,132573,162.6
4,132582,182.9


In [88]:
weight_uniques_train = filtered_train_X.groupby('RecordID')['Weight'].first().reset_index() 
weight_uniques_train.head()

,RecordID,Weight
0,132543,84.6
1,132548,87.0
2,132570,102.6
3,132573,90.1
4,132582,82.5


<h3>Calculate the BMI</h3>

In [89]:
def classify_BMI(BMI):
    if BMI < 18.5:
        return "Baixo peso"
    elif BMI >= 18.6 and BMI <= 24.9:
        return "Peso normal"
    elif BMI >= 25 and BMI <= 29.9:
        return "Sobrepeso"
    elif BMI >= 30 and BMI <= 34.9:
        return "Obesidade grau 1"
    elif BMI >= 35 and BMI <= 39.9:
        return "Obesidade grau 2"
    elif BMI >= 40:
        return "Obesidade grau 3"

In [90]:
height_uniques_train['Height'] = height_uniques_train['Height'] / 100 # Converting Height from cm to meters

In [91]:
bmi_data_train = pd.DataFrame(columns=['RecordID', 'Height', 'Weight', 'BMI', 'Classificacao'])
bmi_data_train["RecordID"] = height_uniques_train["RecordID"]
bmi_data_train["Height"] = height_uniques_train["Height"]
bmi_data_train["Weight"] = weight_uniques_train["Weight"]
bmi_data_train["BMI"] = weight_uniques_train["Weight"] / (height_uniques_train["Height"] ** 2)
bmi_data_train["Classificacao"] = bmi_data_train["BMI"].apply(classify_BMI)
bmi_data_train.head()
# Creating a new DataFrame to store relevant data
#bmi_data = filtered_train_X[['RecordID', 'Height', 'Weight', 'BMI']].reset_index(drop=True)

#bmi_data = pd.concat([height_uniques_train['RecordID'], height_uniques_train['Height'], weight_uniques_train['Weight']], sort=True)
#bmi_data = pd.merge(height_uniques_train[['RecordID', 'Height']], weight_uniques_train[['RecordID', 'Weight']], on='RecordID')
#bmi_data.head()
# bmi_data['BMI'] = weight_uniques_train['Weight'] / (height_uniques_train['Height'] ** 2)


,RecordID,Height,Weight,BMI,Classificacao
0,132543,1.803,84.6,26.024291,Sobrepeso
1,132548,1.626,87.0,32.906233,Obesidade grau 1
2,132570,1.702,102.6,35.418344,Obesidade grau 2
3,132573,1.626,90.1,34.078754,Obesidade grau 1
4,132582,1.829,82.5,24.661900,Peso normal


<h3>Percentage of BMI classification groups</h3>

In [92]:
bmi_data_train["Classificacao"].value_counts(normalize=True) * 100

Classificacao
Sobrepeso           34.648457
Peso normal         29.438543
Obesidade grau 1    18.462317
Obesidade grau 2     7.587253
Obesidade grau 3     6.752656
Baixo peso           3.110774
Name: proportion, dtype: float64

<h2>Validation data</h2>

<h3>Loading validation dataset </h3>

In [93]:
validation_X = physionet2012_dataset['val_X']

<h3>Divided into subgroups by gender and showing the percentage</h3>

In [94]:
distribution_gender_validation = validation_X['Gender'].value_counts(normalize=True) * 100
distribution_gender_validation # 0: female, or 1: male

Gender
 1.0    56.152242
 0.0    43.743483
-1.0     0.104275
Name: proportion, dtype: float64

<h3>Divided into subgroups by ICUType and showing the percentage</h3>

In [95]:
distribution_ICUType_validation = validation_X['ICUType'].value_counts(normalize=True) * 100
distribution_ICUType_validation

ICUType
3.0    36.392075
4.0    27.893639
2.0    20.698644
1.0    15.015641
Name: proportion, dtype: float64

<h3>Filtering only one age per patient</h3>

In [96]:
age_uniques_validation = validation_X.groupby('RecordID')['Age'].first().reset_index()
age_uniques_validation.head()

,RecordID,Age
0,132540,76.0
1,132555,74.0
2,132567,71.0
3,132575,78.0
4,132597,66.0


<h3>Showing total occurrences of age</h3>

In [97]:
age_uniques_validation.count()

RecordID    1918
Age         1918
dtype: int64

<h3>Dividing age into subgroup of people aged 65+</h3>

In [123]:
more_than_or_equal_to_65_validation = age_uniques_validation[age_uniques_validation['Age'] >= 65]
percentage_group_one_validation = (more_than_or_equal_to_65_validation.count() / age_uniques_validation.count())*100
percentage_group_one_validation

RecordID    55.161627
Age         55.161627
dtype: float64

<h3>Filtering only one height and one weight per patient</h3>

In [99]:
filtered_validation_X = validation_X[(validation_X['Height'] != -1) & (validation_X['Weight'] != -1) & (validation_X['Height'].notna()) & (validation_X['Weight'].notna())] 

In [100]:
height_uniques_validation = filtered_validation_X.groupby('RecordID')['Height'].first().reset_index() 
height_uniques_validation.head()

,RecordID,Height
0,132540,175.3
1,132555,175.3
2,132567,157.5
3,132575,167.6
4,132597,137.2


In [101]:
weight_uniques_validation = filtered_validation_X.groupby('RecordID')['Weight'].first().reset_index() 
weight_uniques_validation.head()

,RecordID,Weight
0,132540,76.0
1,132555,66.1
2,132567,56.0
3,132575,63.0
4,132597,82.0


In [102]:
height_uniques_validation['Height'] = height_uniques_validation['Height'] / 100 # Converting Height from cm to meters

In [ ]:
bmi_data_validation = pd.DataFrame(columns=['RecordID', 'Height', 'Weight', 'BMI', 'Classificacao'])
bmi_data_validation["RecordID"] = height_uniques_validation["RecordID"]
bmi_data_validation["Height"] = height_uniques_validation["Height"]
bmi_data_validation["Weight"] = weight_uniques_validation["Weight"]
bmi_data_validation["BMI"] = weight_uniques_validation["Weight"] / (height_uniques_validation["Height"] ** 2)
bmi_data_validation["Classificacao"] = bmi_data_validation["BMI"].apply(classify_BMI)
bmi_data_validation.head()

,RecordID,Height,Weight,BMI,Classificacao
0,132543,1.803,84.6,26.024291,Sobrepeso
1,132548,1.626,87.0,32.906233,Obesidade grau 1
2,132570,1.702,102.6,35.418344,Obesidade grau 2
3,132573,1.626,90.1,34.078754,Obesidade grau 1
4,132582,1.829,82.5,24.661900,Peso normal


<h3>Percentage of BMI classification groups</h3>

In [104]:
bmi_data_validation["Classificacao"].value_counts(normalize=True) * 100

Classificacao
Sobrepeso           34.648457
Peso normal         29.438543
Obesidade grau 1    18.462317
Obesidade grau 2     7.587253
Obesidade grau 3     6.752656
Baixo peso           3.110774
Name: proportion, dtype: float64

<h2>Test data</h2>

In [109]:
test_X = physionet2012_dataset['test_X']

<h3>Divided into subgroups by gender and showing the percentage</h3>


In [106]:
distribution_gender_test = test_X['Gender'].value_counts(normalize=True) * 100
distribution_gender_test # 0: female, or 1: male

Gender
 1.0    55.356398
 0.0    44.518549
-1.0     0.125052
Name: proportion, dtype: float64

<h3>Divided into subgroups by ICUType and showing the percentage</h3>


In [ ]:
distribution_ICUType_test= test_X['ICUType'].value_counts(normalize=True) * 100
distribution_ICUType_test

ICUType
3.0    36.265110
4.0    29.303877
2.0    21.133806
1.0    13.297207
Name: proportion, dtype: float64

<h3>Filtering only one age per patient</h3>


In [111]:
age_uniques_test = test_X.groupby('RecordID')['Age'].first().reset_index()
age_uniques_test.head()

,RecordID,Age
0,132545,88.0
1,132547,64.0
2,132551,78.0
3,132568,66.0
4,132577,65.0


<h3>Showing total occurrences of age</h3>

In [113]:
age_uniques_test.count()

RecordID    2399
Age         2399
dtype: int64

<h3>Dividing age into subgroup of people aged 65+</h3>


In [114]:
more_than_or_equal_to_65_test = age_uniques_test[age_uniques_test['Age'] >= 65]
# more_than_or_equal_to_65 = train_X[train_X['Age'] >= 65] 
#distribution_age_group_one = more_than_or_equal_to_65['Age'].count() 
#distribution_age_group_one
# age_uniques_more_than_or_equal_to_65 = more_than_or_equal_to_65.groupby('RecordID')['Age'].first().reset_index()
percentage_group_one_test = (more_than_or_equal_to_65_test.count() / age_uniques_test.count())*100
percentage_group_one_test

RecordID    53.897457
Age         53.897457
dtype: float64

<h3>Dividing age into subgroup of people under 65 years of age</h3>


In [116]:
less_than_65_test = age_uniques_test[age_uniques_test['Age'] < 65]
# less_than_65 = train_X[train_X['Age'] < 65] 
#distribution_age_group_two = less_than_65['Age'].value_counts(normalize=True) * 100
#distribution_age_group_two

# age_uniques_less_than_65 = less_than_65.groupby('RecordID')['Age'].first().reset_index()
percentage_group_two_test = (less_than_65_test.count() / age_uniques_test.count()) * 100
percentage_group_two_test

RecordID    46.102543
Age         46.102543
dtype: float64

<h3>Filtering only one height and one weight per patient</h3>


In [117]:
filtered_test_X = test_X[(test_X['Height'] != -1) & (test_X['Weight'] != -1) & (test_X['Height'].notna()) & (test_X['Weight'].notna())] 

In [118]:
height_uniques_test = filtered_test_X.groupby('RecordID')['Height'].first().reset_index() 
height_uniques_test.head()

,RecordID,Height
0,132547,180.3
1,132551,162.6
2,132568,157.5
3,132585,165.1
4,132601,177.8


In [119]:
weight_uniques_test = filtered_test_X.groupby('RecordID')['Weight'].first().reset_index() 
weight_uniques_test.head()

,RecordID,Weight
0,132547,114.0
1,132551,48.4
2,132568,84.5
3,132585,84.7
4,132601,75.9


<h3>Calculate the BMI</h3>


In [120]:
height_uniques_test['Height'] = height_uniques_test['Height'] / 100 # Converting Height from cm to meters

In [121]:
bmi_data_test = pd.DataFrame(columns=['RecordID', 'Height', 'Weight', 'BMI', 'Classificacao'])
bmi_data_test["RecordID"] = height_uniques_test["RecordID"]
bmi_data_test["Height"] = height_uniques_test["Height"]
bmi_data_test["Weight"] = weight_uniques_test["Weight"]
bmi_data_test["BMI"] = weight_uniques_test["Weight"] / (height_uniques_test["Height"] ** 2)
bmi_data_test["Classificacao"] = bmi_data_test["BMI"].apply(classify_BMI)
bmi_data_test.head()
# Creating a new DataFrame to store relevant data
#bmi_data = filtered_train_X[['RecordID', 'Height', 'Weight', 'BMI']].reset_index(drop=True)

#bmi_data = pd.concat([height_uniques_train['RecordID'], height_uniques_train['Height'], weight_uniques_train['Weight']], sort=True)
#bmi_data = pd.merge(height_uniques_train[['RecordID', 'Height']], weight_uniques_train[['RecordID', 'Weight']], on='RecordID')
#bmi_data.head()
# bmi_data['BMI'] = weight_uniques_train['Weight'] / (height_uniques_train['Height'] ** 2)


,RecordID,Height,Weight,BMI,Classificacao
0,132547,1.803,114.0,35.068194,Obesidade grau 2
1,132551,1.626,48.4,18.306456,Baixo peso
2,132568,1.575,84.5,34.063996,Obesidade grau 1
3,132585,1.651,84.7,31.073435,Obesidade grau 1
4,132601,1.778,75.9,24.009232,Peso normal


<h3>Percentage of BMI classification groups</h3>

In [122]:
bmi_data_test["Classificacao"].value_counts(normalize=True) * 100


Classificacao
Sobrepeso           34.295136
Peso normal         30.255565
Obesidade grau 1    17.724650
Obesidade grau 3     7.584501
Obesidade grau 2     7.337181
Baixo peso           2.802968
Name: proportion, dtype: float64